<a href="https://colab.research.google.com/github/Dusein/MachineLearningTask/blob/main/6WeekTask/Clustering_Model_For_Online_Retail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

# Masukin path dataset
file_path = '/content/Online Retail.xlsx'
retail_data = pd.read_excel(file_path)

retail_data



,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [6]:
# Data preprocessing for clustering

# Hapus baris dengan CustomerID yang hilang karena sangat penting untuk segmentasi pelanggan
retail_data = retail_data.dropna(subset=['CustomerID']).copy()  # Menambahkan .copy() untuk menghindari SettingWithCopyWarning

# Ubah Tanggal Faktur menjadi tanggal waktu
retail_data.loc[:, 'InvoiceDate'] = pd.to_datetime(retail_data['InvoiceDate'])

# Hitung jumlah total yang dibelanjakan per transaksi
retail_data.loc[:, 'TotalAmount'] = retail_data['Quantity'] * retail_data['UnitPrice']


# Mengagregasi data untuk mendapatkan metrik RFM (Recency, Frequency, Monetary) untuk setiap pelanggan
# Baru-baru ini: Hari sejak pembelian terakhir
# Frekuensi: Jumlah transaksi
# Moneter: Total pengeluaran

# Tentukan tanggal referensi sebagai tanggal transaksi terakhir dalam dataset
reference_date = retail_data['InvoiceDate'].max()

# Aggregate data
rfm_data = retail_data.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',                                     # Frequency
    'TotalAmount': 'sum'                                        # Monetary
}).rename(columns={'InvoiceDate': 'Recency',
                   'InvoiceNo': 'Frequency',
                   'TotalAmount': 'Monetary'})

# Menampilkan beberapa baris pertama dari data RFM
rfm_data.head()


,Recency,Frequency,Monetary
CustomerID,,,
12346.0,325,2,0.00
12347.0,1,7,4310.00
12348.0,74,4,1797.24
12349.0,18,1,1757.55
12350.0,309,1,334.40


In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Skala data RFM untuk kinerja pengelompokan yang lebih baik
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_data)

# Sesuaikan model K-Means
# Memilih jumlah cluster secara acak, katakanlah 4, untuk pengujian awal
kmeans = KMeans(n_clusters=4, random_state=42)
rfm_data['Cluster'] = kmeans.fit_predict(rfm_scaled)

# Menampilkan data RFM yang dikelompokkan dengan penetapan cluster
rfm_data.head()


,Recency,Frequency,Monetary,Cluster
CustomerID,,,,
12346.0,325,2,0.00,1
12347.0,1,7,4310.00,3
12348.0,74,4,1797.24,3
12349.0,18,1,1757.55,3
12350.0,309,1,334.40,1
